In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import torch
import torch.optim as opt
from torch.utils.data import DataLoader

import dataloader
import CTLSTM
import utils

In [43]:
path = 'data_hawkesinhib/'
train = pd.read_pickle(path + 'train.pkl')

In [44]:
train['train'][17]

[{'idx_event': 1,
  'time_since_last_event': 0.107713595,
  'time_since_last_same_event': 0.107713595,
  'type_event': 2,
  'time_since_start': 0.107713595},
 {'idx_event': 2,
  'time_since_last_event': 0.150091,
  'time_since_last_same_event': 0.2578046,
  'type_event': 0,
  'time_since_start': 0.2578046},
 {'idx_event': 3,
  'time_since_last_event': 0.048787028,
  'time_since_last_same_event': 0.19887803,
  'type_event': 2,
  'time_since_start': 0.30659163},
 {'idx_event': 4,
  'time_since_last_event': 0.056226283,
  'time_since_last_same_event': 0.3628179,
  'type_event': 4,
  'time_since_start': 0.3628179},
 {'idx_event': 5,
  'time_since_last_event': 0.1026808,
  'time_since_last_same_event': 0.15890709,
  'type_event': 2,
  'time_since_start': 0.46549872},
 {'idx_event': 6,
  'time_since_last_event': 0.09216979,
  'time_since_last_same_event': 0.2998639,
  'type_event': 0,
  'time_since_start': 0.5576685},
 {'idx_event': 7,
  'time_since_last_event': 0.49327248,
  'time_since_las

In [45]:
settings = {
        'hidden_size': 32,
        'type_size': 8,
        'train_path': path+'train.pkl',
        'dev_path': path+'dev.pkl',
        'batch_size': 32,
        'epoch_num': 100,
        'current_date': datetime.date.today()
    }

In [46]:
"""Training process."""
hidden_size = settings['hidden_size']
type_size = settings['type_size']
train_path = settings['train_path']
dev_path = settings['dev_path']
batch_size = settings['batch_size']
epoch_num = settings['epoch_num']
current_date = settings['current_date']


In [47]:

model = CTLSTM.CTLSTM(hidden_size, type_size)
optim = opt.Adam(model.parameters())
train_dataset = dataloader.CTLSTMDataset(train_path)
dev_dataset = dataloader.CTLSTMDataset(dev_path)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=dataloader.pad_batch_fn, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, collate_fn=dataloader.pad_batch_fn, shuffle=True)


In [49]:

last_dev_loss = 0.0
for epoch in range(epoch_num):
    tic_epoch = time.time()
    epoch_train_loss = 0.0
    epoch_dev_loss = 0.0
    train_event_num = 0
    dev_event_num = 0
    print('Epoch.{} starts.'.format(epoch))
    tic_train = time.time()
    for i_batch, sample_batched in enumerate(train_dataloader):
        tic_batch = time.time()
        
        optim.zero_grad()

        event_seqs, time_seqs, total_time_seqs, seqs_length = utils.pad_bos(sample_batched, model.type_size)

        sim_time_seqs, sim_index_seqs = utils.generate_sim_time_seqs(time_seqs, seqs_length)

        model.forward(event_seqs, time_seqs)
        likelihood = model.log_likelihood(event_seqs, sim_time_seqs, sim_index_seqs, total_time_seqs, seqs_length)
        batch_event_num = torch.sum(seqs_length)
        batch_loss = -likelihood

        batch_loss.backward()
        optim.step()

        toc_batch = time.time()
        if i_batch % 100 == 0:
            print('Epoch.{} Batch.{}:\nBatch Likelihood per event: {:5f} nats\nTrain Time: {:2f} s'.format(epoch, i_batch, likelihood/batch_event_num, toc_batch-tic_batch))
        epoch_train_loss += batch_loss
        train_event_num += batch_event_num

    toc_train = time.time()
    print('---\nEpoch.{} Training set\nTrain Likelihood per event: {:5f} nats\nTrainig Time:{:2f} s'.format(epoch, -epoch_train_loss/train_event_num, toc_train-tic_train))

    tic_eval = time.time()
    for i_batch, sample_batched in enumerate(dev_dataloader):
        event_seqs, time_seqs, total_time_seqs, seqs_length = utils.pad_bos(sample_batched, model.type_size)
        sim_time_seqs, sim_index_seqs = utils.generate_sim_time_seqs(time_seqs, seqs_length)
        model.forward(event_seqs, time_seqs)
        likelihood = model.log_likelihood(event_seqs, sim_time_seqs, sim_index_seqs, total_time_seqs,seqs_length)

        dev_event_num += torch.sum(seqs_length)
        epoch_dev_loss -= likelihood

    toc_eval = time.time()
    toc_epoch = time.time()
    print('Epoch.{} Devlopment set\nDev Likelihood per event: {:5f} nats\nEval Time:{:2f}s.\n'.format(epoch, -epoch_dev_loss/dev_event_num, toc_eval-tic_eval))

    with open("loss_{}.txt".format(current_date), 'a') as l:
        l.write("Epoch {}:\n".format(epoch))
        l.write("Train Event Number:\t\t{}\n".format(train_event_num))
        l.write("Train Likelihood per event:\t{:.5f}\n".format(-epoch_train_loss/train_event_num))
        l.write("Training time:\t\t\t{:.2f} s\n".format(toc_train-tic_train))
        l.write("Dev Event Number:\t\t{}\n".format(dev_event_num))
        l.write("Dev Likelihood per event:\t{:.5f}\n".format(-epoch_dev_loss/dev_event_num))
        l.write("Dev evaluating time:\t\t{:.2f} s\n".format(toc_eval-tic_eval))
        l.write("Epoch time:\t\t\t{:.2f} s\n".format(toc_epoch-tic_epoch))
        l.write("\n")

    gap = epoch_dev_loss/dev_event_num - last_dev_loss
    if abs(gap) < 1e-4:
        print('Final log likelihood: {} nats'.format(-epoch_dev_loss/dev_event_num))
        break

    last_dev_loss = epoch_dev_loss/dev_event_num

Epoch.0 starts.
Epoch.0 Batch.0:
Batch Likelihood per event: -1.709947 nats
Train Time: 0.168332 s
Epoch.0 Batch.100:
Batch Likelihood per event: -1.332159 nats
Train Time: 0.173443 s
Epoch.0 Batch.200:
Batch Likelihood per event: -1.263551 nats
Train Time: 0.174111 s
---
Epoch.0 Training set
Train Likelihood per event: -1.360795 nats
Trainig Time:42.072094 s
Epoch.0 Devlopment set
Dev Likelihood per event: -1.265771 nats
Eval Time:14.086263s.

Epoch.1 starts.
Epoch.1 Batch.0:
Batch Likelihood per event: -1.321297 nats
Train Time: 0.193245 s
Epoch.1 Batch.100:
Batch Likelihood per event: -1.276540 nats
Train Time: 0.170621 s
Epoch.1 Batch.200:
Batch Likelihood per event: -1.260781 nats
Train Time: 0.167851 s
---
Epoch.1 Training set
Train Likelihood per event: -1.260215 nats
Trainig Time:41.637645 s
Epoch.1 Devlopment set
Dev Likelihood per event: -1.258229 nats
Eval Time:14.196925s.

Epoch.2 starts.
Epoch.2 Batch.0:
Batch Likelihood per event: -1.245070 nats
Train Time: 0.192994 s
Epo

Epoch.18 Batch.100:
Batch Likelihood per event: -1.234480 nats
Train Time: 0.162661 s
Epoch.18 Batch.200:
Batch Likelihood per event: -1.156528 nats
Train Time: 0.164647 s
---
Epoch.18 Training set
Train Likelihood per event: -1.205659 nats
Trainig Time:42.528869 s
Epoch.18 Devlopment set
Dev Likelihood per event: -1.205388 nats
Eval Time:14.022093s.

Epoch.19 starts.
Epoch.19 Batch.0:
Batch Likelihood per event: -1.156918 nats
Train Time: 0.189010 s
Epoch.19 Batch.100:
Batch Likelihood per event: -1.218098 nats
Train Time: 0.163393 s
Epoch.19 Batch.200:
Batch Likelihood per event: -1.206109 nats
Train Time: 0.164413 s
---
Epoch.19 Training set
Train Likelihood per event: -1.204360 nats
Trainig Time:42.565656 s
Epoch.19 Devlopment set
Dev Likelihood per event: -1.205138 nats
Eval Time:14.121449s.

Epoch.20 starts.
Epoch.20 Batch.0:
Batch Likelihood per event: -1.169665 nats
Train Time: 0.187171 s
Epoch.20 Batch.100:
Batch Likelihood per event: -1.216623 nats
Train Time: 0.166310 s
Epoc

Epoch.36 Batch.200:
Batch Likelihood per event: -1.196657 nats
Train Time: 0.159768 s
---
Epoch.36 Training set
Train Likelihood per event: -1.189037 nats
Trainig Time:41.864885 s
Epoch.36 Devlopment set
Dev Likelihood per event: -1.190600 nats
Eval Time:13.818538s.

Epoch.37 starts.
Epoch.37 Batch.0:
Batch Likelihood per event: -1.183143 nats
Train Time: 0.195570 s
Epoch.37 Batch.100:
Batch Likelihood per event: -1.153502 nats
Train Time: 0.161474 s
Epoch.37 Batch.200:
Batch Likelihood per event: -1.161895 nats
Train Time: 0.160588 s
---
Epoch.37 Training set
Train Likelihood per event: -1.188445 nats
Trainig Time:41.895195 s
Epoch.37 Devlopment set
Dev Likelihood per event: -1.189085 nats
Eval Time:14.093733s.

Epoch.38 starts.
Epoch.38 Batch.0:
Batch Likelihood per event: -1.172555 nats
Train Time: 0.166993 s
Epoch.38 Batch.100:
Batch Likelihood per event: -1.215709 nats
Train Time: 0.163653 s
Epoch.38 Batch.200:
Batch Likelihood per event: -1.143878 nats
Train Time: 0.171158 s
---


In [10]:
event_seqs

tensor([[   8,   43,   14, 1047,   96, 1896,   15, 3748, 2338, 1145, 1174, 1368,
           20],
        [   8, 3190, 1007,   83,  300,  708,  825, 1533,  184, 2894, 3186,    0,
            0],
        [   8,  593,  198,   48,   66,   44,  425,  242,  548,  545,    0,    0,
            0],
        [   8,  360, 2763,  613,  208,  301,  586,  577,  704,    0,    0,    0,
            0],
        [   8,  396,    3,    5,    6,   60,   46,    0,    0,    0,    0,    0,
            0],
        [   8,  448, 1774,  314, 1827, 2310,    0,    0,    0,    0,    0,    0,
            0],
        [   8,  484,  854,  856,  320,   50,    0,    0,    0,    0,    0,    0,
            0],
        [   8,    2,   13,  127,  612,  169,    0,    0,    0,    0,    0,    0,
            0],
        [   8,   64, 1985,  142,  195,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   8,   86,  111,  202,  713,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   8,  958, 1200, 191

In [11]:
time_seqs

tensor([[0.0000e+00, 3.7500e-01, 8.1250e-01, 4.5625e+00, 1.9688e+00, 1.2500e-01,
         2.5281e+01, 8.1250e-01, 4.0062e+01, 5.6000e+01, 5.0000e-01, 1.4544e+02,
         1.4300e+02],
        [0.0000e+00, 1.2000e+01, 7.7188e+00, 1.1906e+01, 1.7188e+00, 1.2500e+00,
         1.1250e+00, 5.3125e-01, 1.8750e+00, 6.9688e+00, 3.2444e+02, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 9.3750e-02, 1.5625e-01, 9.3750e-02, 0.0000e+00, 3.1250e-02,
         6.2500e-01, 1.2500e-01, 8.7500e-01, 2.1125e+01, 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.8438e+00, 1.2500e-01, 0.0000e+00,
         9.3750e-02, 5.2812e+00, 3.7500e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 3.1250e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.8750e-01,
         6.5938e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 4.3156e+01, 1.4375e+00, 1.6875e+00, 9.0625e-01, 1.3438e+02,